## Install Python dependencies

In [1]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 191.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 308.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 297.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 216.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 296.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 276.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 230.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 288.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 306.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 186.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 252.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 208.2 MB/s eta 0:00:00
     ━━━━━━━━━━

Import the dependencies for the model training code:

In [2]:
import numpy as np
import pandas as pd
import datetime
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
import tf2onnx
import onnx
import pickle
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

2024-09-19 11:13:29.847970: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-19 11:13:30.154311: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-19 11:13:30.413231: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-19 11:13:30.931461: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-19 11:13:31.042497: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-19 11:13:31.212012: I tensorflow/core/platform/cpu_feature_gu

In [3]:
#Laoding the dataset
Data = pd.read_csv('caloric_exp.csv')
Data.head()

,user_id,date,day,age,gender,weight,height_in_cms,digestive_disorder,type_of_workout,additional_activity,...,followed_diet_plan,taking_supplements,fitness_goal,target_weight,target_time_in_months,actual_calorie_intake_in_kcal_per_day,actual_carb_intake_in_gms,actual_protein_intake_in_gms,actual_fat_intake_in_gms,recommended_calorie_intake_in_kcal_per_day
0,1002,2022-01-03,Monday,37,Male,86,154,No,Light Weight Training,Zumba,...,No,No,Weight Loss,6,2,2370,296,118,79,2296
1,1002,2022-01-04,Tuesday,37,Male,86,154,No,Calesthenics,Zumba,...,No,No,Weight Loss,6,2,1965,245,98,65,1906
2,1002,2022-01-05,Wednesday,37,Male,86,154,No,Calesthenics,Zumba,...,Yes,No,Weight Loss,6,2,1695,211,84,56,1848
3,1002,2022-01-06,Thursday,37,Male,86,154,No,Rest,NaN,...,Yes,No,Weight Loss,6,2,1956,244,97,65,2136
4,1002,2022-01-07,Friday,37,Male,86,154,No,Light Weight Training,Zumba,...,No,No,Weight Loss,6,2,2141,267,107,71,2083


In [4]:
# Set the input (X) and output (Y) data.

X = Data.drop(columns = ['user_id','date','recommended_calorie_intake_in_kcal_per_day'])
y = Data['recommended_calorie_intake_in_kcal_per_day']


In [5]:
# Split the data into training and testing sets so you have something to test the trained model with.
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size = 0.2)

In [6]:
Path("artifact").mkdir(parents=True, exist_ok=True)
with open("artifact/test_data.pkl", "wb") as handle:
    pickle.dump((X_test, y_test), handle)

In [7]:
#fetching the cols:
num_cols = X_train.select_dtypes(include='number').columns.to_list()
cat_cols = X_train.select_dtypes(exclude='number').columns.to_list()

In [8]:
#encoding the categorical_variables:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(drop='first',handle_unknown='ignore',sparse_output=False)

In [9]:
#importing the scaler
scaler = StandardScaler()

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

ct = ColumnTransformer(
    transformers = [('Encoder',OneHotEncoder(drop='first',handle_unknown='ignore',sparse_output=False),cat_cols),
                    ('Scaler',StandardScaler(),num_cols)],
                    remainder='passthrough')


In [31]:
ct_01= ct.fit(X_train)

In [32]:
with open("artifact/column_transformer.pkl", "wb") as handle:
    pickle.dump(ct_01, handle)

## Build the model

The model is a simple, fully-connected, deep neural network, containing three hidden layers and one output layer.

In [12]:
X_train_new = ct.fit_transform(X_train)

In [13]:
model = Sequential()
model.add(Dense(32, activation = 'relu', input_dim = X_train_new.shape[1]))
# model.add(Dropout(0.2))
# model.add(Dense(32))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Dropout(0.2))
# model.add(Dense(32))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Dropout(0.2))
model.add(Dense(1, activation = 'linear'))
model.compile(optimizer='adam',loss='mean_squared_error',metrics=['r2_score'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,185 (4.63 KB)

 Trainable params: 1,185 (4.63 KB)

 Non-trainable params: 0 (0.00 B)

## Train the model

Training a model is often the most time-consuming part of the machine learning process.  Large models can take multiple GPUs for days.  Expect the training on CPU for this very simple model to take a minute or more.

In [15]:
# Train the model and get performance
import os

epochs = 200
history = model.fit(X_train_new, y_train.values, epochs=epochs, \
                    validation_data=(ct.transform(X_val),y_val), \
                    verbose = True)
print("Training of model is complete")

Epoch 1/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 5384427.5000 - r2_score: -41.9357 - val_loss: 5270675.0000 - val_r2_score: -42.7327
Epoch 2/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5248821.5000 - r2_score: -42.4517 - val_loss: 5050319.5000 - val_r2_score: -40.9043
Epoch 3/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5015870.5000 - r2_score: -38.5687 - val_loss: 4683843.0000 - val_r2_score: -37.8635
Epoch 4/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4604520.0000 - r2_score: -36.6765 - val_loss: 4218434.0000 - val_r2_score: -34.0019
Epoch 5/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4101503.0000 - r2_score: -32.0222 - val_loss: 3682675.2500 - val_r2_score: -29.5565
Epoch 6/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3552860.0000 - r2_score: -27.8072 - val_loss: 3099851.5000 - val_r2_score: -24.7206
Epoch 7/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2966506.0000 - r2_score: -22.8510 - val_loss: 2526038.7500 - val_r2_s

## Save the model file

In [16]:
os.makedirs("models/caloric/1", exist_ok=True)

In [17]:
saved_model_path = "models/caloric/1/my_model"
model.export(saved_model_path)

INFO:tensorflow:Assets written to: models/caloric/1/my_model/assets


INFO:tensorflow:Assets written to: models/caloric/1/my_model/assets


Saved artifact at 'models/caloric/1/my_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 35), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  139776691596800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139776683029776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139776623920928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139776623920576: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [18]:
!mo --saved_model_dir models/caloric/1/my_model --output_dir models/caloric/1 --framework tf

[ INFO ] MO command line tool is considered as the legacy conversion API as of OpenVINO 2023.2 release.
In 2025.0 MO command line tool and openvino.tools.mo.convert_model() will be removed. Please use OpenVINO Model Converter (OVC) or openvino.convert_model(). OVC represents a lightweight alternative of MO and provides simplified model conversion API. 
Find more information about transition from MO to OVC at https://docs.openvino.ai/2023.2/openvino_docs_OV_Converter_UG_prepare_model_convert_model_MO_OVC_transition.html
[ INFO ] Generated IR will be compressed to FP16. If you get lower accuracy, please consider disabling compression explicitly by adding argument --compress_to_fp16=False.
Find more information about compression to FP16 at https://docs.openvino.ai/2023.0/openvino_docs_MO_DG_FP16_Compression.html
[ SUCCESS ] Generated IR version 11 model.
[ SUCCESS ] XML file: /opt/app-root/src/calorie/models/caloric/1/saved_model.xml
[ SUCCESS ] BIN file: /opt/app-root/src/calorie/models/

In [28]:
X_test.shape

(2000, 20)

In [42]:
X_test.dropna().sample(5).to_csv('X_test.csv',index = False)

In [46]:
X_test.columns

Index(['day', 'age', 'gender', 'weight', 'height_in_cms', 'digestive_disorder',
       'type_of_workout', 'additional_activity', 'total_calories_burned',
       'body_temperature', 'workout_intensity', 'followed_diet_plan',
       'taking_supplements', 'fitness_goal', 'target_weight',
       'target_time_in_months', 'actual_calorie_intake_in_kcal_per_day',
       'actual_carb_intake_in_gms', 'actual_protein_intake_in_gms',
       'actual_fat_intake_in_gms'],
      dtype='object')

In [45]:
X_test.iloc[0,:].values

array(['Tuesday', 30, 'Female', 43, 175, 'No', 'Calesthenics', nan, 272,
       37, 'Low', 'Yes', 'No', 'Weight Gain', 17, 5, 1821, 227, 91, 60],
      dtype=object)